# Day Trading Bot 


## Introduction

#### Leveraging IEX Cloud, yfinance and a trading bot the project emphasies on creating a day trading bot. Data provided for this project by [IEX Cloud](https://iexcloud.io), [Yahoo Finance](https://pypi.org/project/yfinance/)


#### The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.


## Library Imports

#### We need to import the open-source software libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from yahoo_fin.stock_info import get_data

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [29]:
from secrets import IEX_CLOUD_API_TOKEN

## Importing List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis. [Yfinance](https://pypi.org/project/yfinance) was used to gather the data.

#### Let's import the stocks

In [68]:
dow_list = si.tickers_dow()
type(dow_list)

list

In [181]:
dow_list

['AAPL',
 'AMGN',
 'AXP',
 'BA',
 'CAT',
 'CRM',
 'CSCO',
 'CVX',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'V',
 'VZ',
 'WBA',
 'WMT']

In [15]:
print("Tickers in Dow Jones:", len(dow_list))
dow_list[0:10]

Tickers in Dow Jones: 30


['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW']

## Making Our First API Call

Structuring our api call to IEX

We'll need the following information from the API:

* Name of the stock
* Price
* Market capitalization for each stock
* Number of shares to buy
* IEX Close
* IEX Open
* Previous Close
* Previous Volume
* Actual Close
* Close Time



In [11]:
ticker_list = ["amzn", "aapl", "ba"]
historical_datas = {}
for ticker in ticker_list:
    historical_datas[ticker] = get_data(ticker)

In [170]:
dow_historical = {}
for ticker in dow_list:
    dow_historical[ticker] = si.get_data(ticker, start_date="01/01/2020", end_date="01/03/2020", interval="1d")
dow_historical

{'AAPL':                  open       high        low      close   adjclose     volume  \
 2020-01-02  74.059998  75.150002  73.797501  75.087502  74.444603  135480400   
 
            ticker  
 2020-01-02   AAPL  ,
 'AMGN':              open        high         low       close    adjclose   volume  \
 2020-01-02  243.0  243.190002  238.979996  240.100006  233.642029  2088000   
 
            ticker  
 2020-01-02   AMGN  ,
 'AXP':                   open        high         low       close    adjclose  \
 2020-01-02  124.660004  126.269997  124.230003  125.849998  123.637741   
 
              volume ticker  
 2020-01-02  2708000    AXP  ,
 'BA':                   open        high         low       close    adjclose  \
 2020-01-02  328.549988  333.350006  327.700012  333.320007  331.348572   
 
              volume ticker  
 2020-01-02  4544400     BA  ,
 'CAT':              open        high         low       close    adjclose   volume  \
 2020-01-02  149.0  150.550003  147.979996  150.5

In [177]:
symbol

['AAPL',
 'AMGN',
 'AXP',
 'BA',
 'CAT',
 'CRM',
 'CSCO',
 'CVX',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'V',
 'VZ',
 'WBA',
 'WMT']

## Parsing API Call


In [178]:
data['latestPrice']

148.58

## Adding stocks to pandas

In [179]:
for s in symbol[0:4]:
    print(s)
    

AAPL
AMGN
AXP
BA


In [180]:
for s in symbol:
    final_dataframe = final_dataframe.append(
                                        pd.Series(
                                            [ 
                                                   s, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A',
                                                    data['iexClose'],
                                                    data['iexOpen'],
                                                    data['previousClose'],
                                                    data['previousVolume'],
                                                    data['close'],
                                                    data['closeTime']
                                                        ],
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy,IEX Close,IEX Open,Previous Close,Previous Volume,Actual Close,Close Time
0,AAPL,146.890,2458399439852,N/A,146.200,142.650,145.18,102195464,146.16,1643429361884
1,AMGN,251.130,150380377887,N/A,260.080,258.500,264.40,2037132,255.87,1676829984794
2,AXP,116.290,95143061343,N/A,114.850,115.990,117.55,9237452,117.31,1682137363571
3,BA,199.360,114172947773,N/A,196.671,194.376,207.10,11133552,197.08,1628562926994
4,CAT,184.700,101911206043,N/A,182.780,181.560,190.10,2586258,186.00,1637476734510
5,CRM,226.160,206687648397,N/A,227.550,226.420,230.16,5024105,226.42,1635471586710
6,CSCO,47.790,201869161876,N/A,46.270,45.750,47.03,13519663,46.70,1657979716358
7,CVX,91.600,169128379471,N/A,92.100,89.000,92.45,8149579,92.10,1638419377847
8,DIS,168.100,308509673283,N/A,166.920,167.420,175.54,7718037,168.98,1619197071286
9,DOW,56.920,40857220645,N/A,54.980,56.410,56.49,3146695,55.76,1683465650411


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [155]:
dow_list

['AAPL',
 'AMGN',
 'AXP',
 'BA',
 'CAT',
 'CRM',
 'CSCO',
 'CVX',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'V',
 'VZ',
 'WBA',
 'WMT']

In [171]:
historical_datas["aapl"]

,open,high,low,close,adjclose,volume,ticker
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.101087,469033600.0,AAPL
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095813,175884800.0,AAPL
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088780,105728000.0,AAPL
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090978,86441600.0,AAPL
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093615,73449600.0,AAPL
1980-12-19,0.126116,0.126674,0.126116,0.126116,0.099329,48630400.0,AAPL
1980-12-22,0.132254,0.132812,0.132254,0.132254,0.104163,37363200.0,AAPL
1980-12-23,0.137835,0.138393,0.137835,0.137835,0.108558,46950400.0,AAPL
1980-12-24,0.145089,0.145647,0.145089,0.145089,0.114272,48003200.0,AAPL
1980-12-26,0.158482,0.159040,0.158482,0.158482,0.124820,55574400.0,AAPL


In [158]:
final_dataframe = pd.DataFrame(columns = my_columns)
symbol = dow_list

for s in symbol:
    
        api_url = f'https://sandbox.iexapis.com/stable/stock/{s}/quote?token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()
        final_dataframe = final_dataframe.append(
                                pd.Series( 
                                        [
                                         s, 
                                         data['latestPrice'], 
                                         data['marketCap'], 
                                           'N/A',
                                         data['iexClose'],
                                         data['iexOpen'],
                                         data['previousClose'],
                                         data['previousVolume'],
                                         data['close'],
                                         data['closeTime']
                                                         ] ,
                                                          index = my_columns), 
                                            ignore_index = True)


In [174]:
type(data)

dict

In [173]:
final_dataframe.shape

(60, 10)

## Using Batch API Calls to Improve Performance

We will split our list of stocks into groups of 5

In [148]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [153]:
symbol_groups = list(chunks(final_dataframe['Ticker'], 5))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])




In [152]:
final_dataframe = pd.DataFrame(columns = my_columns)

    batch_api_call_url = f'https://sandbox.iexapis.com/stable/{symbol_string}/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                       pd.Series([ 
                                                   symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A',
                                                    data['iexClose'],
                                                    data['iexOpen'],
                                                    data['previousClose'],
                                                    data['previousVolume'],
                                                    data['close'],
                                                    data['closeTime']
                                                        ],
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

IndentationError: unexpected indent (<ipython-input-152-717a04d9113b>, line 3)

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [ ]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

In [ ]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )
```

In [ ]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()

In [21]:
daily_range = si.get_quote_table("aapl")["Day's Range"]
daily_low = float(daily_range.split(" - ")[0])
daily_high = float(daily_range.split(" - ")[1])
print("daily low:", daily_low)
print("daily high:", daily_high)

daily low: 141.37
daily high: 144.3
